In [ ]:
%cd ../

In [ ]:
"""
This is just a simple client example. Hack it as much as you want. 
"""
import argparse
import requests
import jsonpickle
import logging
from PIL import Image, ImageDraw, ImageFont
import pickle
import numpy as np

logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

from glob import glob
from tqdm.notebook import tqdm

import collections
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer
from utils import prepare_device, update_lr_scheduler


# fix random seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

# config = ConfigParser.from_args(args, options)

In [ ]:
url_face = "http://127.0.0.1:10002/"
url_age_gender = "http://127.0.0.1:10003/"

MAXIMUM_ENTROPY = {"gender": 0.6931471805599453, "age": 4.615120516841261}


for image_path in tqdm(glob("./test-images/*") + glob("./test-images-dangerous/*")):
    if "ANNOTATED" in image_path:
        continue
    logging.debug(f"{image_path} loading image ...")
    with open(image_path, "rb") as stream:
        binary_image = stream.read()

    data = {"image": binary_image}
    logging.info(f"image loaded!")

    logging.debug(f"sending image to server...")
    data = jsonpickle.encode(data)
    response = requests.post(url_face, json=data)
    logging.info(f"got {response} from server!...")
    response = jsonpickle.decode(response.text)

    face_detection_recognition = response["face_detection_recognition"]
    logging.info(f"{len(face_detection_recognition)} faces deteced!")

    bboxes = [fdr["bbox"] for fdr in face_detection_recognition]
    det_scores = [fdr["det_score"] for fdr in face_detection_recognition]
    landmarks = [fdr["landmark"] for fdr in face_detection_recognition]

    logging.debug(f"sending embeddings to server ...")
    data = [fdr["normed_embedding"] for fdr in face_detection_recognition]

    # -1 accounts for the batch size.
    data = np.array(data).reshape(-1, 512).astype(np.float32)

    # I wanna get rid of this pickling part but dunno how.
    data = pickle.dumps(data)

    data = {"embeddings": data}
    data = jsonpickle.encode(data)
    response = requests.post(url_age_gender, json=data)
    logging.info(f"got {response} from server!...")

    response = jsonpickle.decode(response.text)
    ages = response["ages"]
    genders = response["genders"]

    logging.debug(f"annotating image ...")
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("fonts/arial.ttf", 25)

    for gender, age, bbox in zip(genders, ages, bboxes):
        draw.rectangle(bbox.tolist(), outline=(0, 0, 0))
        draw.text(
            (bbox[0], bbox[1]),
            f"AGE: {round(age['mean'])} (entropy: {round(age['entropy'], 3)} / {round(MAXIMUM_ENTROPY['age'], 3)})",
            fill=(255, 0, 0),
            font=font,
        )
        draw.text(
            (bbox[0], bbox[3]),
            "MALE " + str(round(gender["m"] * 100)) + str("%") + ", "
            "FEMALE "
            + str(round(gender["f"] * 100))
            + str("%")
            + f" (entropy: {round(gender['entropy'], 3)} / {round(MAXIMUM_ENTROPY['gender'], 3)})",
            fill=(0, 255, 0),
            font=font,
        )
        image.save(image_path + ".ANNOTATED.jpg")
    logging.debug(f"image annotated and saved at {image_path + '.ANNOTATED.jpg'}")